In [1]:
%matplotlib notebook

import os
import time
import numpy as np
import matplotlib as mpl

from collections import OrderedDict
from importlib import reload
from matplotlib import pyplot as plt

In [2]:
import qcodes as qc
from qcodes import load_by_id
from qcodes import validators
from qcodes.dataset.measurements import Measurement
from qcodes.instrument.base import Instrument
from qcodes.instrument.parameter import Parameter

In [3]:
from pytopo.qctools.instruments import create_inst
from pytopo.qctools.dataset2 import select_experiment

from plottr import client
from plottr.qcodes_dataset import QcodesDatasetSubscriber

from pytopo.mplplots.init_nb_plotting import *

qc.config['core']['db_location'] = r"d:/OneDrive/BF1/Data/experiments.db"

In [4]:
from qcodes.instrument_drivers.QuTech.IVVI import IVVI
ivvi = create_inst(IVVI, "ivvi", "ASRL5::INSTR")

from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
key1 = create_inst(Keysight_34465A, "key1", "TCPIP::169.254.4.62::INSTR", force_new_instance=True)
key2 = create_inst(Keysight_34465A, "key2", "TCPIP::169.254.4.61::INSTR", force_new_instance=True)
key3 = create_inst(Keysight_34465A, "key3", "TCPIP::169.254.21.92::INSTR", force_new_instance=True)
key4 = create_inst(Keysight_34465A, "key4", "TCPIP::169.254.130.42::INSTR", force_new_instance=True)

from qcodes.instrument_drivers.stanford_research.SR860 import SR860
lockin1 = create_inst(SR860, "lockin1", "GPIB0::3::INSTR")
lockin2 = create_inst(SR860, "lockin2", "GPIB0::4::INSTR")
lockin3 = create_inst(SR860, "lockin3", "GPIB0::5::INSTR")

for i in range(1,16):
    ivvi.parameters['dac{}'.format(i)].set_step(0.5)
    ivvi.parameters['dac{}'.format(i)].set_delay(0.001)
    
ivvi.dac1.set_delay(0.001)
ivvi.dac1.set_step(20)

ivvi.dac2.set_delay(0.001)
ivvi.dac2.set_step(20)

ivvi.dac3.set_delay(0.001)
ivvi.dac3.set_step(2)

ivvi.dac4.set_delay(0.001)
ivvi.dac4.set_step(20)

ivvi.dac5.set_delay(0.001)
ivvi.dac5.set_step(20)

ivvi.dac6.set_delay(0.001)
ivvi.dac6.set_step(2)




from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430,AMI430_3D
ami_x = create_inst(AMI430, "AMI430_x", "169.254.3.64", port = 7180, has_current_rating=True)
ami_y = create_inst(AMI430, "AMI430_y", "169.254.54.62", port = 7180, has_current_rating=True)
ami_z = create_inst(AMI430, "AMI430_z", "169.254.73.58", port = 7180, has_current_rating=True)

#ramp_limit_multiplier = 0.6
#ami_x.current_ramp_limit(0.1742*ramp_limit_multiplier)
#ami_x.coil_constant(0.01491)
#ami_x.current_rating(67.07)

#ami_y.current_ramp_limit(0.1762*ramp_limit_multiplier)
#ami_y.coil_constant(0.01490)
#ami_y.current_rating(67.11)

#ami_z.current_ramp_limit(0.0756*ramp_limit_multiplier)
#ami_z.coil_constant(0.0834)
#ami_z.current_rating(71.94)

ami = create_inst(AMI430_3D, "AMI430", ami_x, ami_y, ami_z, 2)




Initialized IVVI-rack in 0.08s
Connected to: Keysight Technologies 34465A (serial:MY57503135, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.12s
Connected to: Keysight Technologies 34465A (serial:MY57503556, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.12s
Connected to: Keysight Technologies 34465A (serial:MY57503596, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.17s
Connected to: Keysight Technologies 34465A (serial:MY57504203, firmware:A.02.16-02.40-02.16-00.51-03-01) in 0.13s
Connected to: Stanford_Research_Systems SR860 (serial:003104, firmware:V1.47) in 0.10s
Connected to: Stanford_Research_Systems SR860 (serial:003102, firmware:V1.47) in 0.05s
Connected to: Stanford_Research_Systems SR860 (serial:003106, firmware:V1.47) in 0.03s


d:\onedrive\bf1\code\qcodes\qcodes\instrument\parameter.py:518: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "
d:\onedrive\bf1\code\qcodes\qcodes\instrument\parameter.py:530: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "


Connected to: AMERICAN MAGNETICS INC. 430 (serial:170025, firmware:2.50) in 0.99s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170026, firmware:2.50) in 0.82s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170024, firmware:2.50) in 1.03s


In [5]:
ami.x(0.0)
ami.y(0.0)
ami.z(0.0)

In [12]:
ami.spherical([0.0,90,90])

In [13]:
ami.spherical()

[0.0, 90.0, 90.0]

In [107]:
ivvi.set_dacs_zero()

In [110]:
class ConversionParameter(Parameter):
    
    def __init__(self, name, src_param, get_conv, set_conv=None, **kw):
        super().__init__(name, **kw)
        self.src_param = src_param
        self.get_conv = get_conv
        self.set_conv = set_conv
    
    def get_raw(self):
        return self.get_conv(self.src_param())
    
    def set_raw(self, value):
        if self.set_conv is None:
            raise NotImplementedError("No set conversion implemented.")
        
        return self.src_param(self.set_conv(value))
    

class IVVISetup(Instrument):
    
    def __init__(self, name, ivvi, **kw):
        super().__init__(name, **kw)
        
        self.ivvi = ivvi
        
        self.add_parameter('voltage_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=1e-3)
        self.add_parameter('current_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=1e-7)
        self.add_parameter('iso_in_multiplier', get_cmd=None, set_cmd=None,
                           initial_value=1e-2)
        self.add_parameter('gate_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=15)
        
        self.add_parameter('M1b_gain', get_cmd=None, set_cmd=None, initial_value=1e6)
        self.add_parameter('M1b_mode', get_cmd=None, set_cmd=None, 
                           vals=validators.Enum('low_noise', 'low_R_in'), 
                           initial_value='low_noise')
        self.add_parameter('M1b_R_in', get_cmd=self._get_M1b_R_in)
        
        self.add_parameter('M2m_Vgain', get_cmd=None, set_cmd=None, initial_value=1e3)

        for i in range(1, self.ivvi._numdacs + 1):       
            self.add_parameter('dac{}'.format(i), unit='V',
                               get_cmd=self.ivvi._gen_ch_get_func(self._get_dac, i),
                               set_cmd=self.ivvi._gen_ch_set_func(self._set_dac, i))
            
    def _get_M1b_R_in(self):
        if self.M1b_mode() == 'low_noise':
            return 2e3 + 1e-3 * self.M1b_gain()
        elif self.M1b_mode() == 'low_R_in':
            return 2e3 + 1e-4 * self.M1b_gain()
            
    def _get_dac(self, ch):
        if ch == 6:
            return self.ivvi.set('dac{}'.format(ch)) * 1e-3 * self.gate_multiplier()
        else:
            return self.ivvi.get('dac{}'.format(ch)) * 1e-3 * self.voltage_multiplier()
    
    def _set_dac(self, ch, val):
        if ch == 6:
            return self.ivvi.set('dac{}'.format(ch), val * 1e3 / self.gate_multiplier())
        else:
            return self.ivvi.set('dac{}'.format(ch), val * 1e3 / self.voltage_multiplier())
    




In [133]:
ivvi_setup = create_inst(IVVISetup, 'ivvi_setup', ivvi=ivvi, force_new_instance=True)
#ivvi_setup.voltage_multiplier(10e-3)
ivvi_setup.voltage_multiplier(100e-9)
ivvi_setup.gate_multiplier(15)
ivvi_setup.M2m_Vgain(1000)


dev1_line_r = 20e3 + 3.4e3 #1.7e3 from Qfilter, 5e3 for DC-only on PCB, 10e3 for RF on PCB, 3e3 for 8x MUX bias-T
dev2_line_r = 15e3 + 3.4e3
dev3_line_r = 15e3 + 3.4e3

dev1_rs = dev1_line_r + ivvi_setup.M1b_R_in()
dev2_rs = dev2_line_r + ivvi_setup.M1b_R_in()
dev3_rs = dev3_line_r + ivvi_setup.M1b_R_in()

dev1_Vac = lockin1.amplitude()*ivvi_setup.iso_in_multiplier()*ivvi_setup.voltage_multiplier()
dev2_Vac = lockin1.amplitude()*ivvi_setup.iso_in_multiplier()*ivvi_setup.voltage_multiplier()
dev3_Vac = lockin1.amplitude()*ivvi_setup.iso_in_multiplier()*ivvi_setup.voltage_multiplier()

dev1_Iac = lockin1.amplitude()*ivvi_setup.iso_in_multiplier()*ivvi_setup.current_multiplier()
dev2_Iac = lockin2.amplitude()*ivvi_setup.iso_in_multiplier()*ivvi_setup.current_multiplier()

ivvi_setup.add_parameter('v1_measurement', parameter_class=ConversionParameter, unit='V',
                          src_param=key1.volt, get_conv=lambda x: x/ivvi_setup.M2m_Vgain())
ivvi_setup.add_parameter('v2_measurement', parameter_class=ConversionParameter, unit='V',
                          src_param=key2.volt, get_conv=lambda x: x/(ivvi_setup.M2m_Vgain()))
ivvi_setup.add_parameter('i1_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key1.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())
ivvi_setup.add_parameter('i2_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key2.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())
ivvi_setup.add_parameter('i3_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key3.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())
ivvi_setup.add_parameter('ig_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key4.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())

ivvi_setup.add_parameter('g1_measurement', parameter_class=ConversionParameter, unit='2e2/h',
                         src_param=lockin1.X, get_conv=lambda x: 12906/(dev1_Vac*ivvi_setup.M1b_gain()/x - dev1_rs))
ivvi_setup.add_parameter('g2_measurement', parameter_class=ConversionParameter, unit='2e2/h',
                         src_param=lockin2.X, get_conv=lambda x: 12906/(dev2_Vac*ivvi_setup.M1b_gain()/x - dev2_rs))
ivvi_setup.add_parameter('g3_measurement', parameter_class=ConversionParameter, unit='2e2/h',
                         src_param=lockin3.X, get_conv=lambda x: 12906/(dev3_Vac*ivvi_setup.M1b_gain()/x - dev3_rs))

ivvi_setup.add_parameter('r_ac1_measurement', parameter_class=ConversionParameter, unit='Ohms',
                         src_param=lockin1.X, get_conv=lambda x: x/(dev1_Iac*ivvi_setup.M2m_Vgain()))
ivvi_setup.add_parameter('r_ac2_measurement', parameter_class=ConversionParameter, unit='Ohms',
                         src_param=lockin2.X, get_conv=lambda x: x/(dev2_Iac*ivvi_setup.M2m_Vgain()))

Instrument ivvi_setup already exists.


In [7]:
ivvi_setup.M1b_gain(1e6)
ivvi_setup.M1b_mode('low_noise')

key1.NPLC(1)
key2.NPLC(1)
key3.NPLC(1)

print('R_in: ', ivvi_setup.M1b_R_in())
print('Rs_1: ', dev1_rs)
print('Rs_2: ', dev2_rs)
print('Rs_3: ', dev3_rs)

R_in:  3000.0
Rs_1:  16400.0
Rs_2:  21400.0
Rs_3:  21400.0


## Pinchoff curves

In [62]:
ivvi_setup.dac5(10e-3)

In [26]:
exp = select_experiment('V0_0112A', 'DEV_51')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac6)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ivvi_setup.dac6, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac6, ])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ivvi_setup.dac6, ])


ivvi_setup.dac5(10e-3)
srcvals = np.linspace(6, -10, 401)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac6(s)
        datasaver.add_result((ivvi_setup.dac6, s), 
                             (ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                             #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                             #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()))#,
                             #(ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                             #(ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 295


In [27]:
exp = select_experiment('V0_0112A', 'DEV_51')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac6)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ivvi_setup.dac6, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac6, ])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ivvi_setup.dac6, ])


ivvi_setup.dac5(10e-3)
srcvals = np.linspace(-10, 6, 401)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac6(s)
        datasaver.add_result((ivvi_setup.dac6, s), 
                             (ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                             #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                             #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()))#,
                             #(ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                             #(ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 296


In [1]:
ivvi.set_dacs_zero()

NameError: name 'ivvi' is not defined

## Bias vs gate

In [106]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
#meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac5])

lockin1.amplitude(0.1)

srcvals = np.linspace(6, -10, 1000)
bias_values = np.linspace(-.8e-3, .8e-3, 121)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in srcvals:
        ivvi_setup.dac6(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ivvi_setup.dac6, s1),
                                 (ivvi_setup.dac5, s2), 
                                 (ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 (ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 (ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 317


KeyboardInterrupt: 

## Bias vs theta scans

In [42]:
ami.spherical([0.3,0,0])

In [167]:
ami.z_measured()

1.3e-06

In [44]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.theta)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])

lockin1.amplitude(0.1)

theta_vals = np.linspace(0, 180, 90)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 301


In [45]:
ami.spherical([0.3,180,150])

In [46]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.theta)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])

lockin1.amplitude(0.1)

theta_vals = np.linspace(180, 0, 90)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 302


In [47]:
ami.spherical([0.3,0,30])

In [48]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.theta)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])

lockin1.amplitude(0.1)

theta_vals = np.linspace(0, 180, 90)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 303


In [49]:
ami.spherical([0.3,180,60])

In [50]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.theta)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])

lockin1.amplitude(0.1)

theta_vals = np.linspace(180, 0, 90)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 304


In [51]:
ami.spherical([0.3,0,90])

In [54]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.theta)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])

lockin1.amplitude(0.1)

theta_vals = np.linspace(0, 180, 90)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 306


In [55]:
ami.spherical([0.3,180,120])

In [56]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.theta)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.theta, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.theta, ivvi_setup.dac5])

lockin1.amplitude(0.1)

theta_vals = np.linspace(180, 0, 90)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 307


In [57]:
ami.spherical([0,180,120])

## Bias vs Br sweep

In [58]:
ami.spherical([0,17,0])

In [61]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.field, ivvi_setup.dac5])

lockin1.amplitude(0.1)

Br_vals = np.linspace(0, 1, 101)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in Br_vals:
        ami.field(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 308


In [62]:
ami.spherical([1,17,150])

In [63]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.field, ivvi_setup.dac5])

lockin1.amplitude(0.1)

Br_vals = np.linspace(1, 0, 101)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in Br_vals:
        ami.field(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 309


In [64]:
ami.spherical([0,69,0])

In [65]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.field, ivvi_setup.dac5])

lockin1.amplitude(0.1)

Br_vals = np.linspace(0, 1, 101)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in Br_vals:
        ami.field(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 310


In [73]:
x = 1*np.sin(62*np.pi/180)*np.cos(150*np.pi/180)
y = 1*np.sin(62*np.pi/180)*np.sin(150*np.pi/180)
z = 1*np.cos(62*np.pi/180)
print('Bx set: ' + str(x))
print('By set: ' + str(y))
print('Bz set: ' + str(z))

Bx set: -0.764655045626
By set: 0.441473796429
Bz set: 0.469471562786


In [90]:
print('Bx measured: ' + str(ami.x_measured()))
print('By measured: ' + str(ami.y_measured()))
print('Bz measured: ' + str(ami.z_measured()))

Bx measured: 0.3462617
By measured: -4.2e-06
Bz measured: 0.3583391


In [87]:
ami_x.pause()

In [91]:
ami.spherical([1,62,150])

In [92]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.field, ivvi_setup.dac5])

lockin1.amplitude(0.1)

Br_vals = np.linspace(1, 0, 101)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in Br_vals:
        ami.field(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 311


In [93]:
ami.spherical([0,17,90])

In [94]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.field, ivvi_setup.dac5])

lockin1.amplitude(0.1)

Br_vals = np.linspace(0, 1, 101)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in Br_vals:
        ami.field(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 312


In [95]:
ami.spherical([1,35,90])

In [96]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.field, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.field, ivvi_setup.dac5])

lockin1.amplitude(0.1)

Br_vals = np.linspace(1, 0, 101)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in Br_vals:
        ami.field(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 313


## Bias vs phi sweeps

In [101]:
ami.spherical([0.6,17,0])

In [102]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.phi)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.phi, ivvi_setup.dac5])

lockin1.amplitude(0.1)

phi_vals = np.linspace(0, 180, 91)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in phi_vals:
        ami.phi(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.phi, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 315


In [103]:
ami.spherical([0.6,35,180])

In [104]:
exp = select_experiment('V0_0112A', 'DEV_30_33_35')

meas = Measurement(exp=exp)
meas.write_period = 1
#meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.phi)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
#meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
#meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ami.phi, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ami.phi, ivvi_setup.dac5])

lockin1.amplitude(0.1)

phi_vals = np.linspace(180, 0, 91)
bias_values = np.linspace(-.6e-3, .6e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in phi_vals:
        ami.phi(s1)
        for s2 in bias_values:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.phi, s1),
                                 (ivvi_setup.dac5, s2), 
                                 #(ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                                 #(ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                                 #(ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                                 #(ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                                 (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                                 (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

Starting experimental run with id: 316


In [105]:
ami.field(0)

In [206]:
ivvi_setup.dac4(0)

## I bias sweep

In [205]:
exp = select_experiment('V0_0112A', 'DEV_29_NC')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ivvi_setup.v1_measurement, setpoints=[ivvi_setup.dac4, ])
meas.register_parameter(ivvi_setup.v2_measurement, setpoints=[ivvi_setup.dac4, ])
meas.register_parameter(ivvi_setup.r_ac1_measurement, setpoints=[ivvi_setup.dac4, ])
meas.register_parameter(ivvi_setup.r_ac2_measurement, setpoints=[ivvi_setup.dac4, ])
#meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ivvi_setup.dac6, ])



srcvals = np.linspace(-25e-9, 25e-9, 301)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac4(s)
        time.sleep(0.2)
        datasaver.add_result((ivvi_setup.dac4, s), 
                             (ivvi_setup.v1_measurement, ivvi_setup.v1_measurement()),
                             (ivvi_setup.v2_measurement, ivvi_setup.v2_measurement()),
                             (ivvi_setup.r_ac1_measurement, ivvi_setup.r_ac1_measurement()),
                             (ivvi_setup.r_ac2_measurement, ivvi_setup.r_ac2_measurement()))

Starting experimental run with id: 354


## I bias vs gate

In [165]:
exp = select_experiment('V0_0112A', 'DEV_37_45')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.v1_measurement, setpoints=[ivvi_setup.dac4, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.v2_measurement, setpoints=[ivvi_setup.dac4, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.r_ac1_measurement, setpoints=[ivvi_setup.dac4, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.r_ac2_measurement, setpoints=[ivvi_setup.dac4, ivvi_setup.dac6])
#meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ivvi_setup.dac6, ])
#meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ivvi_setup.dac6, ])



srcvals = np.linspace(-100e-9, 100e-9, 601)
gatevals = np.linspace(10,0,101)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in gatevals:
        ivvi_setup.dac6(s)
        for s2 in srcvals:
            ivvi_setup.dac4(s2)
            #time.sleep(0.2)
            datasaver.add_result((ivvi_setup.dac4, s2), 
                                 (ivvi_setup.dac6, s),
                                 (ivvi_setup.v1_measurement, ivvi_setup.v1_measurement()),
                                 (ivvi_setup.v2_measurement, ivvi_setup.v2_measurement()),
                                 (ivvi_setup.r_ac1_measurement, ivvi_setup.r_ac1_measurement()),
                                 (ivvi_setup.r_ac2_measurement, ivvi_setup.r_ac2_measurement()))

Starting experimental run with id: 338


KeyboardInterrupt: 

In [217]:
ivvi_setup.dac4(0)

In [211]:
ami_z.pause()

## Ibias vs theta scan

In [223]:
ami_x.pause()
ami_y.pause()
ami_z.pause()

In [224]:
ami.spherical([0,41,90])

KeyboardInterrupt: 

In [220]:
exp = select_experiment('V0_0112A', 'DEV_31_34')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ami.theta)
meas.register_parameter(ivvi_setup.v1_measurement, setpoints=[ivvi_setup.dac4, ami.theta])
meas.register_parameter(ivvi_setup.v2_measurement, setpoints=[ivvi_setup.dac4, ami.theta])
# meas.register_parameter(ivvi_setup.r_ac1_measurement, setpoints=[ivvi_setup.dac4, ami.theta])
# meas.register_parameter(ivvi_setup.r_ac2_measurement, setpoints=[ivvi_setup.dac4, ami.theta])

lockin1.amplitude(0.1)



theta_vals = np.linspace(0, 180, 60)
bias_values = np.linspace(-50e-9, 50e-9, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac4(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac4, s2), 
                                 (ivvi_setup.v1_measurement, ivvi_setup.v1_measurement()),
                                 (ivvi_setup.v2_measurement, ivvi_setup.v2_measurement()))
#                                  (ivvi_setup.r_ac1_measurement, ivvi_setup.r_ac1_measurement()),
#                                  (ivvi_setup.r_ac2_measurement, ivvi_setup.r_ac2_measurement()))

Starting experimental run with id: 358


KeyboardInterrupt: 

## I bias vs field magnitude

In [231]:
ami_x.pause()
ami_y.pause()
ami_z.pause()
ami.spherical([0,41,90])
ivvi.set_dacs_zero()

In [230]:
exp = select_experiment('V0_0112A', 'DEV_31_34')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ami.field)
meas.register_parameter(ivvi_setup.v1_measurement, setpoints=[ivvi_setup.dac4, ami.field])
meas.register_parameter(ivvi_setup.v2_measurement, setpoints=[ivvi_setup.dac4, ami.field])
#meas.register_parameter(ivvi_setup.r_ac1_measurement, setpoints=[ivvi_setup.dac4, ami.field])
#meas.register_parameter(ivvi_setup.r_ac2_measurement, setpoints=[ivvi_setup.dac4, ami.field])

lockin1.amplitude(0.1)

Br_vals_fw = np.linspace(0, 1, 41)
Br_vals_bw = np.linspace(0, 1, 41)
bias_values = np.linspace(-.5e-7, .5e-7, 101)
bg_vals = [10]

for s0 in bg_vals: 
    if s0%2 == 0: 
        with meas.run() as datasaver:
            datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                        min_wait=0, min_count=1)

            for s1 in Br_vals_fw:
                ami.field(s1)
                for s2 in bias_values:
                    ivvi_setup.dac4(s2)
                    datasaver.add_result((ami.field, s1),
                                         (ivvi_setup.dac4, s2), 
                                         (ivvi_setup.v1_measurement, ivvi_setup.v1_measurement()),
                                         (ivvi_setup.v2_measurement, ivvi_setup.v2_measurement()))#,
                                         #(ivvi_setup.r_ac1_measurement, ivvi_setup.r_ac1_measurement()),
                                         #(ivvi_setup.r_ac2_measurement, ivvi_setup.r_ac2_measurement()))
    if s0%2 == 1:
        with meas.run() as datasaver:
            datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                        min_wait=0, min_count=1)

            for s1 in Br_vals_bw:
                ami.field(s1)
                for s2 in bias_values:
                    ivvi_setup.dac4(s2)
                    datasaver.add_result((ami.field, s1),
                                         (ivvi_setup.dac4, s2), 
                                         (ivvi_setup.v1_measurement, ivvi_setup.v1_measurement()),
                                         (ivvi_setup.v2_measurement, ivvi_setup.v2_measurement()))#,
                                         #(ivvi_setup.r_ac1_measurement, ivvi_setup.r_ac1_measurement()),
                                         #(ivvi_setup.r_ac2_measurement, ivvi_setup.r_ac2_measurement()))

Starting experimental run with id: 361


In [ ]:
exp = select_experiment('V0_0112A', 'DEV_2_3_4')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
# meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.i1_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.i2_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.i3_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.g1_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.g2_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.g3_measurement, setpoints=[ivvi_setup.dac5, ])


srcvals = np.linspace(-10e-3, 10e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac5(s)
        datasaver.add_result((ivvi_setup.dac5, s), 
                             (ivvi_setup.i1_measurement, ivvi_setup.i1_measurement()),
                             (ivvi_setup.i2_measurement, ivvi_setup.i2_measurement()),
                             (ivvi_setup.i3_measurement, ivvi_setup.i3_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g1_measurement, ivvi_setup.g1_measurement()),
                             (ivvi_setup.g2_measurement, ivvi_setup.g2_measurement()),
                             (ivvi_setup.g3_measurement, ivvi_setup.g3_measurement()))

In [ ]:
ivvi_setup = create_inst(IVVISetup, 'ivvi_setup', ivvi=ivvi, force_new_instance=True)
ivvi_setup.voltage_multiplier(100e-9)

ivvi_setup.add_parameter('v_measurement', parameter_class=ConversionParameter, unit='V',
                         src_param=key2.volt, get_conv=lambda x: x/200)
ivvi_setup.add_parameter('i_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key1.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())

In [ ]:
exp = select_experiment('DEV43 Isw test', 'V0_0112A')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac2)
meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac2, ])
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac2, ])


srcvals = np.linspace(-30e-9, 30e-9, 151)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac2(s)
        datasaver.add_result((ivvi_setup.dac2, s), 
                             (ivvi_setup.v_measurement, ivvi_setup.v_measurement()),
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()))

In [ ]:
ivvi.dac2(100)

In [ ]:
key2.volt()

In [232]:
ivvi.set_dacs_zero()

In [ ]:
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600

In [ ]:
keithley = create_inst(Keithley_2600, "keithley", "USB0::0x05E6::0x2614::4345885::INSTR")

In [ ]:
keithley.smua.res()

In [ ]:
exp = select_experiment('V0_0112A', 'Cooldown')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac1)
meas.register_parameter(ivvi_setup.v_measurement, setpoints=[ivvi_setup.dac1, ])
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac1, ])


srcvals = np.linspace(0, 0, 3e5)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    i = 0 
    for s in srcvals:
        i += 1
#         ivvi_setup.dac1(s)
        time.sleep(1)
        datasaver.add_result((ivvi_setup.dac1, i), 
                             (ivvi_setup.v_measurement, keithley.smua.res()),
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()))

In [ ]:
Br = 2
theta = 17
phi = 90

Bx = Br*np.sin(theta*np.pi/180)*np.cos(phi*np.pi/180)
By = Br*np.sin(theta*np.pi/180)*np.sin(phi*np.pi/180)
Bz = Br*np.cos(theta*np.pi/180)
print('Bx = ' + str(Bx) + ' T')
print('By = ' + str(By) + ' T')
print('Bz = ' + str(Bz) + ' T')